# 実験を開始する

実験記録に必要な実験パッケージの用意によって実験の作成と開始をサポートします。  
実験パッケージは、研究リポジトリのexperimentsフォルダ配下に子関係として追加されます。  
上から順番に実行してください。  
2回目以降の実行の場合、このセルが選択された状態で画面上部に表示される以下のボタンをクリックしてから実行して下さい。  
![UnfreezeBotton](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/sections/images/unfreeze_button.png)

## 1. 開始する実験の実験パッケージ名を入力する

以下のセルを実行し、表示されるテキストボックスに作成したい実験パッケージ名を半角英数字で入力してください。  
データガバナンス機能にすでに存在するリポジトリ名のリポジトリは作成いただけません。実験名など分かりやすい名前をご記入ください。  
入力した実験パッケージ名の履歴をこのノートブックに保存したい場合は、実行後にセルの左側に表示されるピンマークをクリックしてください。  
※入力値に誤りがある場合、もう一度実行することで訂正ができます。

In [ ]:
import re
from IPython.display import clear_output

# GINサーバのものに合わせたバリデーションルールを設定
validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

print('作成したい実験パッケージ名を半角英数字で入力してください。')
while True:
    repository_title = input("パッケージ名：")
    if validation.fullmatch(repository_title):
        break
    else:
        clear_output()
        print('パッケージ名は英数字、および"-", "_", "."のみで入力してください。')

clear_output()
print("作成したパッケージ名：", repository_title)
print('この実験名で処理を進めます。変更したい場合は、このセルをもう一度実行することで訂正ができます。')


In [ ]:
import json
import os

path_flow_root = '/home/jovyan/WORKFLOW/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

## 2. 実験記録管理のための準備を行う

### - 2.1 実験パッケージを用意する

In [ ]:
tmp_path = '/home/jovyan/experiments/' + repository_title + '/'

In [ ]:
%%bash
if [ ! -d $tmp_path ]; then
    # 実験タイトル名のフォルダが無ければ作成する
    mkdir $tmp_path
fi

In [ ]:
import os

os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をパラメタファイルから取得する
monitoring_params = {}
with open(utils.fetch_monitoring_param_file_path(), mode='r') as f:
    monitoring_params = json.load(f)
    
scheme_name = monitoring_params['datasetStructure']

# 実験パッケージを用意する
!cp -r ~/WORKFLOW/PACKAGE/base/. $tmp_path
!cp -r ~/WORKFLOW/PACKAGE/scheme/$scheme_name/. $tmp_path

# dataset化してsiblingを設定する
%cd $tmp_path
remote_http_url = !git config --get remote.origin.url
for item in remote_http_url:
    remote_url = item.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
!datalad create --force .
!datalad siblings add -s "gin" --url $remote_url

In [ ]:
# データガバナンス機能に同期する
import papermill as pm
pm.execute_notebook(
    '/home/jovyan/WORKFLOW/FLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験パッケージの用意', PATH=tmp_path, TO_GIT = True)
)

### - 3.2 pipeline.jsonに実験名を追記する

In [ ]:
import json

with open('/home/jovyan/experiments/pipeline.json', 'r') as f:
    pipeline = json.load(f)

pipeline.append(repository_title)

with open('/home/jovyan/experiments/pipeline.json', 'w') as f:
    json.dump(pipeline, f, indent = 4)

## 4. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import papermill as pm

%cd ~/

pm.execute_notebook(
    'WORKFLOW/FLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験パッケージの作成', IS_RECURSIVE = False, TO_GIT = True)
)

## 5. 実験環境にアクセスする

以下のうち、いずれかの手順を実施してください。

- 標準的な実験活動の場合：「5-A. 標準環境での実験」
- 高性能実験環境を利用する場合：「5-B. 高性能実験環境での実験」

### - 5-A. 標準環境での実験

以下のセルを実行した後に出力されるリンクをクリックして実験に移ってください。

In [ ]:
import urllib

print(
    "https://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(remote_http_url, safe='') + "/HEAD?filepath=experiment.ipynb"
)

### - 5-B. 高性能実験環境での実験

以下のセルを実行し、「高性能実験環境と正常に接続されています」と表示されることを確認してください。  
表示されない場合、[高性能実験環境を準備する](../01_preparation_phase/base_setup_data_analysis_tools.ipynb) の手順を実施してください。

In [ ]:
!ssh mdx "echo 高性能実験環境と正常に接続されています"

高性能実験環境の接続先情報を取得します。以下のセルを実行してください。

In [ ]:
mdx_host = ''
with open('/home/jovyan/.ssh/config', 'r') as f:
    tmp = f.read()
    tmp = tmp[tmp.find('mdx'):]
    tmp = tmp[tmp.find('Hostname ') + len('Hostname '):]
    mdx_host = tmp[:tmp.find('\n')]

高性能実験環境に実験パッケージ情報を転送します。以下のセルを実行してください。

In [ ]:
cmd = "git clone " + remote_http_url
!ssh mdx $cmd

高性能実験環境においてJupyterLabを起動します。以下のセルを実行してください。  
実行後、表示されるリンクをクリックすると、パスワードの入力が求められますので「gpu-jupyter」と入力してLoginしてください。

In [ ]:
from IPython.display import clear_output

# JupyterLabコンテナイメージの作成
cmd = "docker build -t dg/jupyterlab:1.1 ./" + repository_title + "/"
!ssh mdx $cmd
clear_output()

# JupyterLabコンテナの起動
if mdx_host == '163.220.176.51':
    cmd = "docker run --gpus all -d -P dg/jupyterlab:1.1"
else:
    cmd = "docker run -d -P dg/jupyterlab:1.1"
container_id = !ssh mdx $cmd
container_id = container_id[0][:12]

# ホストマシン上で動的に割り当てられたポートを取得
cmd = "docker ps -l"
port = !ssh mdx $cmd
port = port[1]
port = port[port.find("0.0.0.0:") + 8: port.find("->8888/tcp")]

# JupyterLab利用のためのURLを表示
url = "http://" + mdx_host + ":" + str(port) + "/notebooks/experiment.ipynb"
print(url)

## 6. ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../base_FLOW.ipynb)からトップページに遷移できます。  